In [1]:
from utils import dataset
from torchvision import transforms
import numpy as np
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
from Net.resnext import resnext101_32x8d_wsl
import torchvision.models as models
from Net.proposal_layer import RegionProposalNetwork
from utils.get_layer import IntermediateLayerGetter
from Net.roi_pool import VGG16RoIHead

In [25]:
# %%
carpath = "nuimages/train/car.json"
humanpath = "nuimages/train/human.json"
car_imgs = []
car_imgs = dataset.load_image(carpath, 0)

transform = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
print(car_imgs[2]['annotations'])


[{'token': '000091dc8fbc4079b412f735aa743c71', 'category_token': 'fd69059b62a3469fbaef25340c0eab7f', 'bbox': [0, 484, 558, 893], 'mask': {'size': [900, 1600], 'counts': 'aWEwVTptYTAxTzJOMk4yW09kMDAxTzEwME8xTzAwMU8xMDAwTzEwME8xMDAwME8xMDBPMTAwTzEwME8xMDAwME8xMDBPMTAwTzJPMU4xMDFPMU4yTzBPMk9PMDEwTzFPMDAxME8wMU8wMTBPMU8wMDEwTzAxTzAwMTAwTzAwMTBPMDEwME8wMDEwTzAxME8wMTAwTzAxME9OMk00TDNNNEwzTTRMM000TDNNM000TDNNNEwzTTRMM04zTTJNNE0yTjJPMk0yTzJOMU4zTjFGO0Y5MDAwMU8wMDFPME8yTzAwMU8wMDAwMU8wMDFPMDAxMDBPMU8xTzFPMU8wMDFPMTAwTzFPMU8xTzFPMDAxTzEwME8xTzFPMU8wMDFPMU8xMDBPMU8xTzAwMU8xTzFPMDEwTzFPMU8wMDFPMU8xTzAxME8xTzFPMU8wMDFPMU8xME8wMU8xTzFPMDAxMDBPMU8wMDEwME8xTzAxME8xTzFPMTAwTzFPMU8xMDBPMU8xTzEwME8xTzEwME8xTzFPMTAwTzFPMU8xMDBPMU8xMDBPMU8xTzEwME8xTzEwME8xTzEwME8xTzEwMU4xMDBPMU8xMDBPMU8xMDBPMTAwTzFPMTAwTzEwME8xTzEwME8xTzEwME8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzEwME8xTzEwME8xTzEwME8xTzEwME8xTzEwME

In [3]:
car_train = dataset.Dataset(car_imgs, transform=transform, loader=dataset.Myloader)
car_train.__getitem__(1)

img = car_train.__getitem__(0).numpy()
max_val = img.max()
img = np.uint8(img*255/max_val)
img = img.transpose(1,2,0)

In [14]:
batch_size = 2
car_train_data =  DataLoader(dataset=car_train, batch_size=batch_size, shuffle=True, num_workers=0)
Cuda = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#list(enumerate(car_train_data, 0))
#
#resnext_model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl').to(device)
rpn_model = RegionProposalNetwork().cuda()

model = models.vgg16(pretrained=True).cuda()
return_layers = {'features':'Conv2d', 'classifier':'conv'}
backbone = IntermediateLayerGetter(model, return_layers)
ROI_head_model = VGG16RoIHead(n_class=3, roi_size=7,spatial_scale=1/16, classifier=backbone['classifier'][:-3]).cuda()

C:\Users\Jiarun\AppData\Local\anaconda3\envs\ECE449project\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Jiarun\AppData\Local\anaconda3\envs\ECE449project\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
for i, image in enumerate(car_train_data, 0):
    model.eval()
    return_layers = {'features':'Conv2d', }
    backbone = IntermediateLayerGetter(model, return_layers)
    backbone.eval()
    image = Variable(image.to(device))
    with torch.no_grad():
        for j in range(len(backbone['features'])):
            if j==0:
                x = backbone['features'][j](image)
            else:
                x = backbone['features'][j](x)
            if j == 28:
                conv5_3 = x


    # print(conv5_3.size())

In [16]:
rpn_locs, rpn_scores, rois, roi_indices, anchor = rpn_model(conv5_3, (32,56))
# print(roi_indices)
roi_cls_locs, roi_scores = ROI_head_model(conv5_3,rois,roi_indices)
# print(roi_cls_locs)
print(roi_scores)

tensor([[-0.1335, -0.1883,  0.8460],
        [-0.3081,  0.0600,  0.8123],
        [-0.3045, -0.2777,  0.4974],
        [-0.4454,  0.0340,  0.2688],
        [-0.3455, -0.2605,  0.7213],
        [-0.5193, -0.0130,  0.8797],
        [-0.6692, -0.0960,  0.7921],
        [-0.2380, -0.1021,  0.6433],
        [-0.3284, -0.1322,  0.6298],
        [ 0.5677, -0.1135, -0.0235],
        [ 0.4639,  0.1808,  0.0013],
        [-0.0436, -0.1405, -0.1620],
        [ 0.5413,  0.1262,  0.0652],
        [ 0.2848,  0.1643, -0.3401],
        [ 0.4117,  0.4020, -0.3943],
        [ 0.5137,  0.4155,  0.1259],
        [ 0.1120,  0.2362, -0.1431]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [13]:
backbone['classifier'][:-3].eval()

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
)